In [1]:
import pandas as pd
from process import process_stock
from var_lag_order import var_lag_order

In [2]:
stock_list = []
f = open('./VaR/stock_list.txt')
for line in f.readlines():
    line = line.strip('\n')
    stock_list.append(line)

In [3]:
token = '094f15d71394516b730602faa77b1c708007b8d05df300590b4445ed'
starttime = '20180101'
endtime = '20191232'
process = process_stock(stock_list,token)
df = process.create_time_series(starttime, endtime,1000,'limit')
process.save_df(path = './stock2.csv')

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
data = pd.read_csv('stock2.csv', index_col=0)
varlag = var_lag_order(data)
varlag.test_stationary('5%')
print(varlag.column_list)
print(varlag.get_stock_index())

processdata = varlag.process_data

['yield_rate_000001.SZ', 'yield_rate_600015.SH', 'yield_rate_601398.SH', 'yield_rate_601939.SH', 'yield_rate_601988.SH', 'yield_rate_601998.SH', 'amihud_000001.SZ', 'amihud_600015.SH', 'amihud_601398.SH', 'amihud_601939.SH', 'amihud_601988.SH', 'amihud_601998.SH']
[0, 2, 8, 10, 11, 12]


In [5]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.stats.diagnostic
from statsmodels.tsa.api import VAR

In [6]:
model = VAR(processdata)
lag_order = model.select_order(15)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [7]:
lag_order.summary()

,AIC,BIC,FPE,HQIC
0,-193.0,-192.9,1.512e-84,-193.0
1,-194.5,-193.2*,3.296e-85,-194.0*
2,-194.7,-192.0,2.890e-85*,-193.6
3,-194.7*,-190.8,2.893e-85,-193.1
4,-194.7,-189.5,2.938e-85,-192.6
5,-194.6,-188.1,3.189e-85,-192.0
6,-194.4,-186.6,4.036e-85,-191.3
7,-194.2,-185.2,4.873e-85,-190.7
8,-194.1,-183.9,5.387e-85,-190.1
9,-193.9,-182.4,7.026e-85,-189.3


In [17]:
results = model.fit(maxlags = 15, ic='aic')

In [18]:
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 29, Apr, 2021
Time:                     14:00:10
--------------------------------------------------------------------
No. of Equations:         12.0000    BIC:                   -190.739
Nobs:                     484.000    HQIC:                  -193.068
Log likelihood:           39290.0    FPE:                3.15194e-85
AIC:                     -194.575    Det(Omega_mle):     1.30218e-85
--------------------------------------------------------------------
Results for equation yield_rate_000001.SZ
                             coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------------
const                          -0.006523         0.003723           -1.752           0.080
L1.yield_rate_000001.SZ         0.002668         0.080081            0.033           0.973
L1.yi